# Demo for `ObjectSummary.py`
## Import

In [1]:
from OSGenerator import OSGenerator

import json

## Building the graph

In [2]:
osg = OSGenerator()
osg.build("data/friends.txt", "data/checkins.txt")

Building place graph (2/2): 100%|████████████| 6442892/6442892 [01:23<00:00, 77336.56it/s]


In [3]:
for p in osg.graph.edges(1000, keys=True):
    print(p)

(1000, 1, 'friends_with')
(1000, 207, 'friends_with')
(1000, 615, 'friends_with')
(1000, 1395, 'friends_with')
(1000, 4979, 'friends_with')
(1000, 43588, 'friends_with')
(1000, 2071063, 'checksin_at')
(1000, 3340957, 'checksin_at')
(1000, 1946746, 'checksin_at')
(1000, 3335022, 'checksin_at')
(1000, 3333213, 'checksin_at')


In [4]:
osg.graph[1000][1946746]["checksin_at"]["time"]

[datetime.datetime(2010, 7, 14, 15, 22, 5),
 datetime.datetime(2010, 7, 13, 9, 46, 22)]

In [5]:
osg.graph.nodes[1946746]

{'lat': 51.459313, 'lng': 5.3915300833}

## Find all places visited by user `1000`

In [6]:
[x - 1900654 for x in osg.get_checkins(1000)]

[170409, 1440303, 46092, 1434368, 1432559]

## When did `1000` visited `46092` (or `1946746`)
All `place_id`s are shifted by `1900654`

In [7]:
osg.get_visittime(1000, 1946746)

[datetime.datetime(2010, 7, 14, 15, 22, 5),
 datetime.datetime(2010, 7, 13, 9, 46, 22)]

## Find user `1000`'s friends

In [8]:
osg.get_friends(1000)

[1, 207, 615, 1395, 4979, 43588]

## Show information about POI `46092` (or `1946746`)
This gives information about latitude, longitude, as well as users checked in at this place

In [9]:
osg.get_placeinfo(1946746)

{'placeid': 1946746, 'lat': 51.459313, 'lng': 5.3915300833}

## Find POI `46092`'s (or `1946746`) visitors

In [10]:
osg.get_visitor(1946746)

[1000,
 13014,
 16719,
 19889,
 20790,
 27396,
 37388,
 39541,
 45392,
 52605,
 53224,
 63478,
 92068,
 94765,
 97703,
 104088,
 110422,
 112195,
 112813,
 113101,
 120179,
 125152,
 132368,
 132769,
 132780,
 139658,
 142379,
 149669,
 152586,
 153299,
 180505]

## Show OS for user `1000`

In [11]:
os1 = osg.get_object_summary(1000, prettify=True)

{
    "user": 1000,
    "friends": [
        1,
        207,
        615,
        1395,
        4979,
        43588
    ],
    "checksin_at": [
        {
            "placeid": 2071063,
            "time": [
                "16-47-2010 14:07:1279284473"
            ],
            "visitors": [
                1000,
                13014,
                16719,
                19889,
                20790,
                27396,
                37388,
                39541,
                45392,
                52605,
                53224,
                63478,
                92068,
                94765,
                97703,
                104088,
                110422,
                112195,
                112813,
                113101,
                120179,
                125152,
                132368,
                132769,
                132780,
                139658,
                142379,
                149669,
                152586,
                153299,
 

## TODO: define objects' weights and similarity function

### Candidate objects' weight definition
$$o.w_{user = A} = len(o.u|_{u = A})$$

### Candidate similarity function definition
1. Jaccard Coefficient
$$ Sim(o_1, o_2) = \frac{|o_1.users\cap o_2.users|}{|o_1.users\cup o_2.users|}$$